In [12]:
from logging import ERROR
import random
import json
import pickle
import numpy as np
from google.protobuf import message

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from numpy.core.defchararray import mod
from numpy.lib.npyio import load

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


PS: Untuk file 'intents.json' bisa diupload dari device lokal atau dengan menggunakan google drive.

In [13]:
lemmatizer=WordNetLemmatizer()
intents=json.loads(open('drive/MyDrive/chatbot/intents.json').read())

print(intents)

{'intents': [{'tag': 'noanswer', 'patterns': [''], 'responses': ['Maaf, aku tidak mengerti', 'Maaf, aku tidak mengerti. Silahkan ketik ulang', 'Aku tidak yakin apa maksudmu'], 'context': ['']}, {'tag': 'greetings', 'patterns': ['hai', 'halo', 'permisi', 'hei', 'test', 'hi'], 'responses': ['Halo!, apa yang bisa saya bantu?', 'Hai, senang bertemu denganmu! ketik tanya jika kamu ingin bertanya.', 'Hai, ada yang bisa saya bantu?', 'Selamat datang di Peraturanku Bot! Apa yang bisa saya bantu?'], 'context': ['']}, {'tag': 'farewell', 'patterns': ['sampai nanti', 'selamat tinggal', 'sampai jumpa', 'bye'], 'responses': ['Sampai bertemu lain kali!', 'Sampai jumpa!', 'Selamat tinggal :)', 'Sedihnya harus berpisah:('], 'context': ['']}, {'tag': 'thanks', 'patterns': ['trims', 'makasih', 'terimakasih', 'terima kasih', 'terima kasih sudah membantu', 'thanks', 'thank you'], 'responses': ['Senang bisa membantu:)', 'Tentu, tidak masalah', 'Terimakasih kembali!'], 'context': ['']}, {'tag': 'sorry', 'pa

In [14]:
words = []
classes = []
documents= []
ignore_letters = ["?","!",".",","]


for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))

training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training=np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [15]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
CHATBOT_MODEL = "chatbotmodel.h5"
model.save(CHATBOT_MODEL, hist)

print("DONE!")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/200
12/12 [==============================] - 0s 2ms/step - loss: 2.9048 - accuracy: 0.0893
Epoch 2/200
12/12 [==============================] - 0s 2ms/step - loss: 2.8387 - accuracy: 0.0714
Epoch 3/200
12/12 [==============================] - 0s 2ms/step - loss: 2.7999 - accuracy: 0.1071
Epoch 4/200
12/12 [==============================] - 0s 2ms/step - loss: 2.7802 - accuracy: 0.1250
Epoch 5/200
12/12 [==============================] - 0s 2ms/step - loss: 2.6943 - accuracy: 0.1429
Epoch 6/200
12/12 [==============================] - 0s 2ms/step - loss: 2.6280 - accuracy: 0.1607
Epoch 7/200
12/12 [==============================] - 0s 2ms/step - loss: 2.5798 - accuracy: 0.2500
Epoch 8/200
12/12 [==============================] - 0s 2ms/step - loss: 2.4937 - accuracy: 0.2143
Epoch 9/200
12/12 [==============================] - 0s 2ms/step - loss: 2.4160 - accuracy: 0.2143
Epoch 10/200
12/12 [==============================] - 0s 2ms/step - loss: 2.3300 - accuracy: 0.2857
Epoch 11/

In [16]:
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
model = load_model('chatbotmodel.h5')

In [17]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1

    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_TRESHOLD = 0.25
    results = [[i,r]for i, r in enumerate(res) if r > ERROR_TRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]],'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

In [18]:
TF_LITE_CHATBOT_MODEL = "chatbot.tflite"

tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = tf_lite_converter.convert()

open(TF_LITE_CHATBOT_MODEL, "wb").write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpr7v0e1ep/assets


INFO:tensorflow:Assets written to: /tmp/tmpr7v0e1ep/assets


73824

In [19]:
print("================================")
print("         PERATURANKU BOT        ")
print("================================")
print("      Type (stop) for exit!     ")
print(" ")

while True:
    message = input("YOU: ")
    if message.lower() == "stop":
        print("Terimakasih telah menggunakan Peraturanku Bot!")
        break

    ints = predict_class(message)
    res = get_response(ints, intents)
    print("BOT: " + res)

         PERATURANKU BOT        
      Type (stop) for exit!     
 
YOU: hai
BOT: Hai, senang bertemu denganmu! ketik tanya jika kamu ingin bertanya.
YOU: tanya
BOT: Ada yang bisa di bantu?
YOU: apa yang bisa kamu lakukan?
BOT: Aku dapat menjelaskan tentang aplikasi Peraturanku, serta apa saja mengenai peraturan yang ada di aplikasi ini.
YOU: apa itu peraturanku?
BOT: Peraturanku adalah sebuah aplikasi yang berisi tentang Undang-Undang dan Peraturan yang ada di wilayah DKI Jakarta.
YOU: peraturan apa saja yang ada?
BOT: Peraturan Daerah, Peraturan Gubernur dan Keputusan Gubernur.
YOU: cari peraturan daerah
BOT: Untuk memuat Peraturan Daerah silahkan kembali ke Menu Utama, lalu pilih Peraturan Daerah yang ingin kamu baca atau unduh.
YOU: buka keputusan gubernur
BOT: Untuk memuat Keputusan Gubernur silahkan kembali ke Menu Utama, lalu pilih Keputusan Gubernur yang ingin kamu baca atau unduh.
YOU: baiklah
BOT: Oke, apa ada lagi yang dapat saya bantu?
YOU: tidak ada
BOT: Baiklah, untuk men